### CONTENT NOTEBOOK

IMPORTS

VARIABLES

FUNCTIONS

CLASS

MAIN CODE

BUILD AND TESTS

### ----------------------------------------------------------------

### 
IMPORTS

In [1]:
import os
import sys
import json
import pandas
import urllib
import requests
from urllib import parse, request



VARIABLES

In [2]:
REST_URL = "http://data.bioontology.org"
API_KEY = '3b00793b-f3cc-489c-9d6e-7a888f4b656e'
PATH_TO_PAPERS = 'data/kaggle/'
PATH_TO_ONTS_SELECT = 'data/onts_select.csv'



FUNCTIONS

In [3]:
def parse_directory():
    '''For parse and join paths in PATH_TO_PAPERS.
       Output:path_files: list with all paths to record 
    '''
    global path_files
    path_files = []

    for root, dirs, files in os.walk(PATH_TO_PAPERS):
        for file in files:
            path_files.append(os.path.join(root, file))

    return path_files

In [4]:
def get_metadata_record(path_to_record):
    '''Get dict keys from JSON-PAPER data structure.
       Input:path_to_json record.
       Output: record.keys.
    '''
    with open(path_to_record,'r') as json_handler:
        record=json.load(json_handler)
    return record.keys()

In [5]:
def load_json_record(path_to_record):
    ''' Load entire json record for a paper.
        Input: path to record.
        Output: json_record.
    '''
    global json_record
    with open(path_to_record, 'r') as json_handler:
        json_record = json.load(json_handler)
    return json_record

In [6]:
def load_select_onts():
    ''' For reading and proccess ontologies selected file in PATH_TO_ONTS_SELECT.
        Output: select_onts.
    '''
    #proces file.
    ont_sel = pandas.read_csv(PATH_TO_ONTS_SELECT)
    list_onto = [x.replace('\t','').replace(' ','').replace('\n','') for x in list(ont_sel['Ontology'].values)if type(x)!=float]
    #
    global select_onts
    select_onts= []
    for i in list_onto:
        if i not in select_onts:
            select_onts.append(i)
        else:
            next
            
    return select_onts

In [7]:
def get_json(url):
    
    '''
    General util: call some bioportal endpoint.
    Header Auth method.
    Input:general url bioportal.
    Output: json response fo a general urls requests.
    
    '''
    
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)] 
    return json.loads(opener.open(url).read())

In [8]:
def load_bio_onts():
    '''Use bioportal endPoint to retrive all Bioportal Onts'''
    
    resources = get_json(REST_URL + "/")                    # call to retrieve json
    ontologies = get_json(resources["links"]["ontologies"]) # call again for especific fiels of json response
    
    global bio_onts
    bio_onts = []
    for ontology in ontologies:
        bio_onts.append(f"{ontology['name']}\n{ontology['@id']}\n")
        
    return bio_onts    

In [9]:
def onts():
    '''
       Merge select_onts and bio_onts,
       for  build later string-url-params that cotain only ontlogies
       selected and existing in bioportal.
       Output: ontologies url query parameter 
    '''
    
    select_onts = load_select_onts() # call other function
    bio_onts = load_bio_onts()       # call function
   
    
    #build an string characteres that contain a unique words: each word is a 
    # ontologie selected and existing in bio.
    global ontologies
    ontologies = '' 
    
    for ontosel in select_onts:          
        for ontobio in bio_onts:
            if ontobio.find(ontosel) > 1:          # bugOne: the word MESH make match twice and twice is append,
                if ontologies.find(ontosel) < 1:   # but they should be appended only once. i don't catch .
                    ontologies += ontosel +' '     # the reason of this.
                else:
                    pass
    ontologies = ontologies[:-1].replace(' ',',')
    
    return ontologies

In [10]:
onts()

'MESH,MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO,NCIT,CIDO,VO,IOBC,BAO,COVID19,MEDDRA,COVIDCRFRAPID,EFO,CODO'

In [ ]:
def str_list_onts(ontologie_list):
    '''General util.
       make methamorfosis from list to string-of-elements
       Input:ontologies_list
       Output: ontologies url query parameter
    '''
    global ontologies
    ontologies = ""
    for ont in ontologie_list:
        ontologies += ','+ ont
    return ontologies[1:]

#### nota: Es necesario estandarizar el archivo onts_select.csv: 
- se deben registrar las siglas oficiales de cada ontologia registrada. 
- se deben incluir ontologias que existan en bioportal.


para el caso de "TheCOVID-19InfectiousDiseaseOntology" la sigla oficias es IDO-COVID19. vease: https://bioportal.bioontology.org/ontologies/IDO-COVID-19

para el caso de: "NHC" no se encuentra en bioportal ontologias. vease: 
https://bioportal.bioontology.org/ontologies

In [ ]:
ontologies = "MESH,COVID-19,SNOMEDCT,COVID19"

In [11]:
def make_url_query(text_to_annotate):
    '''return URL'''
    global URL 
    URL = REST_URL + "/annotator/?text=%s"%(parse.quote(text_to_annotate))
    return URL

In [12]:
#Call api V.2
def call_endpoint_annotator(URL, PARAMS):
    r = requests.get(url = URL, params = PARAMS)
    global json_api_results
    json_api_results = r.json()
    return json_api_results

### 
CLASS

In [14]:
class dbMain:
    
    '''this class store data for tree schemas.
    s1: item collection.
    s2:chunks of paper
    s2:annotations for each chunk.
    contain methods for update schemas an store db main.
    Methods:
    annotations['matchTerm'].append(matchTerm)
    annotations['ontology'].append(ontology)
    annotations['annotatedClass'].append(annotatedClass)
    chunks['text'].append(text_to_annotate)
    chunks['sections'].append(section_text_to_annotated)
    chunks['chunks_paper'].append(annotations)
    collection['chunks_paper'].append(chunks)
    collection['paper_id'] += paper_id
    db.append(item_collection).
    '''
    
    db = [] # list of items_collections

    def __init__(self):

        self.item_collection = {
                                'paper_id':'',
                                'chunks_paper':[]
                                  }
        
        self.chunks = {
                        'texts':[],
                        'sections':[],
                        'annotations':[]
                        }

        self.annotations = {
                            'matchTerm':[],
                            'ontology':[],
                            'annotatedClass':[]
                            }

    #methods
    def restart_annotations(self):
        self.annotations = {
                            'matchTerm':[],
                            'ontology':[],
                            'annotatedClass':[]
                            }
    def update_annotations(self,matchTerm,
                                ontology,
                                annotatedClass):
        self.annotations['matchTerm'].append(matchTerm)
        self.annotations['ontology'].append(ontology)
        self.annotations['annotatedClass'].append(annotatedClass)
    
    def update_chunks(self,text,section,annotations):
        self.chunks['texts'].append(text)
        self.chunks['sections'].append(section)
        self.chunks['annotations'].append(annotations)

    def update_item_collection(self,paper_id,chunks_paper):
        self.item_collection['paper_id'] += paper_id
        self.item_collection['chunks_paper'].append(chunks_paper)
        
    def update_db_main(self):
        dbMain.db.append(self.item_collection)
        
    def store_db_main(self):
        
        with open('annotationsDBlist.json','w') as json_file:
            json_file.write(json.dumps(dbMain().db))

MAIN CODE

In [62]:
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO,NCIT,CIDO,VO,IOBC,BAO,COVID19,MEDDRA,COVIDCRFRAPID,EFO,CODO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#parse_directory()

parse_directory()

for idx, path in enumerate(path_files):
    
    load_json_record(path)
    emty_sections = []
    item = dbMain()
        
    try: 
        
        for idx, _chunk in enumerate(json_record['abstract']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init db schema annotations
                item.restart_annotations()

                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']
                                           )
                print(item.annotations)
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
    
            else:
                emty_sections.append(idx)

        print('ok abstract')
        #Init api params

        for idx, _chunk in enumerate(json_record['body_text']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                item.restart_annotations()

                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']
                                           )
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
            else:
                emty_sections.append(idx)
                
    except KeyError:

        for idx, _chunk in enumerate(json_record['body_text']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                item.restart_annotations()
                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']

                                           )
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
            else:
                emty_sections.append(idx)

    finally:
        #for title
        text_to_annotate = json_record['metadata']['title']
        
        if text_to_annotate:
            make_url_query(text_to_annotate)
            print(URL)

            #call api here
            call_endpoint_annotator(URL,PARAMS)
            #init schema annotations
            
            item.restart_annotations()
            #proccess and store resultsfor section here
            for i in range(len(json_api_results)):
                item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                        json_api_results[i]['annotatedClass']['links']['ontology'],
                                        json_api_results[i]['annotatedClass']['@id']

                                           )
            item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
        item.update_item_collection(json_record['paper_id'], item.chunks)
        print('paper index:',idx,'processed')
        item.update_db_main()
item.store_db_main()

Mycoplasma pneumoniae is a common cause of upper and lower respiratory tract infections. It remains one of the most frequent causes of atypical pneumonia particularly among young adults.[1,2,3,4,5] Although it is highly transmissible, most infections caused by this organism are relatively minor and include pharyngitis, tracheobronchitis, bronchiolitis, and croup with one fifth of infections being asymptomatic.[6,7] Only 3 -10% of infected subjects develop symptoms consistent with bronchopneumonia and mortality from infection is rare.[6,7] The organism is fastidious and difficult to grow on cultures. Therefore, diagnosis of infections caused by this organism is usually confirmed with serological tests or polymerase chain reaction-gene amplification techniques. At King Abdulaziz University Hospital (KAUH), Jeddah, Saudi Arabia, the facility to perform Mycoplasma culture has been available since January 1997. As published information concerning M. pneumoniae infections in Saudi Arabia is 

Outcome of patients with M. pneumoniae infection was classified into 4 categories; uneventful recovery, recovery following complications, death due to M. pneumoniae infection, or death unrelated to M. pneumoniae infection.
http://data.bioontology.org/annotator/?text=Outcome%20of%20patients%20with%20M.%20pneumoniae%20infection%20was%20classified%20into%204%20categories%3B%20uneventful%20recovery%2C%20recovery%20following%20complications%2C%20death%20due%20to%20M.%20pneumoniae%20infection%2C%20or%20death%20unrelated%20to%20M.%20pneumoniae%20infection.
The Statistical Package for Social Sciences (SPSS) program was used for data analysis. Comparison of categorical data was by Chi-square statistic or Fisher's exact test for small expected values.
http://data.bioontology.org/annotator/?text=The%20Statistical%20Package%20for%20Social%20Sciences%20%28SPSS%29%20program%20was%20used%20for%20data%20analysis.%20Comparison%20of%20categorical%20data%20was%20by%20Chi-square%20statistic%20or%20Fisher%

Mycoplasma pneumoniae is one of the most common causes of atypical pneumonia accounting for 5-23% of community-acquired pneumonia,[1,2,3,4,5] In a study of 511 children with acute respiratory tract infection in Riyadh, Saudi Arabia, Mycoplasma pneumoniae was found to be the second most common causative agent after Respiratory syncytial virus (RSV) accounting for 9% of all cases,[8] In a study of 112 adult patients with community acquired pneumonia admitted to a military hospital in Riyadh, Saudi Arabia, this organism accounted for 6% of all cases,[9] In another retrospective study of 567 pneumonic episodes in adult patients from Al-Qassim area, the organism accounted for 23% of all episodes,[10] The organism also causes other relatively minor infections such as pharyngitis, tracheobronchitis, bronchiolitis, and croup. It is transmitted from person-to-person by infected respiratory droplets during close contact. It is most common in school-aged children, military recruits, and college s

More than half (56.5%) of the patients with pneumonia had uneventful recovery. Mortality from M. pneumoniae pneumonia was high (12.5%) and occurred only in patients with underlying comorbidities. None of the 9 patients with no underlying comorbidities died of M pneumoniae pneumonia. The relatively high complications rate (16.7%) and mortality (12.5%) related to M. pneumoniae pneumonia are likely due to selection bias as most patients with pneumonia were sick enough to require admission to the hospital (21/24 or 87.5%) and most of them had comorbidities (20/24 or 83.3%).
http://data.bioontology.org/annotator/?text=More%20than%20half%20%2856.5%25%29%20of%20the%20patients%20with%20pneumonia%20had%20uneventful%20recovery.%20Mortality%20from%20M.%20pneumoniae%20pneumonia%20was%20high%20%2812.5%25%29%20and%20occurred%20only%20in%20patients%20with%20underlying%20comorbidities.%20None%20of%20the%209%20patients%20with%20no%20underlying%20comorbidities%20died%20of%20M%20pneumoniae%20pneumonia.%2

Sites of extrapulmonary expression have also been described in small mammals. In the rat, SP-D message was identified in RNA extracted from skin and blood vessel [16], and both protein and message were identified in gastric mucosa [17] and mesentery [13]. Using RT-PCR, SP-D message has also been identified in mouse stomach, heart, and kidney [14].
http://data.bioontology.org/annotator/?text=Sites%20of%20extrapulmonary%20expression%20have%20also%20been%20described%20in%20small%20mammals.%20In%20the%20rat%2C%20SP-D%20message%20was%20identified%20in%20RNA%20extracted%20from%20skin%20and%20blood%20vessel%20%5B16%5D%2C%20and%20both%20protein%20and%20message%20were%20identified%20in%20gastric%20mucosa%20%5B17%5D%20and%20mesentery%20%5B13%5D.%20Using%20RT-PCR%2C%20SP-D%20message%20has%20also%20been%20identified%20in%20mouse%20stomach%2C%20heart%2C%20and%20kidney%20%5B14%5D.
Recent crystallographic and mutagenesis studies suggest that the structural determinants of saccharide binding are simil

The N-terminal peptide of the mature protein contains two conserved cysteine residues at positions 15 and 20. These residues participate in interchain disulfide crosslinks that stabilize the trimer, as well as the N-terminal association of four or more trimeric subunits. Stable oligomerization of trimeric subunits permits cooperative or bridging interactions between spatially separated binding sites on the same surface or on different particles.
http://data.bioontology.org/annotator/?text=The%20N-terminal%20peptide%20of%20the%20mature%20protein%20contains%20two%20conserved%20cysteine%20residues%20at%20positions%2015%20and%2020.%20These%20residues%20participate%20in%20interchain%20disulfide%20crosslinks%20that%20stabilize%20the%20trimer%2C%20as%20well%20as%20the%20N-terminal%20association%20of%20four%20or%20more%20trimeric%20subunits.%20Stable%20oligomerization%20of%20trimeric%20subunits%20permits%20cooperative%20or%20bridging%20interactions%20between%20spatially%20separated%20binding%2

SP-D binds to purified lipopolysaccharide (LPS) isolated from a variety of Gram-negative organisms [35,41]. In addition, LPS is the major cell wall component that is labeled on lectin blotting of outer membranes isolated from Escherichia coli [41]. The latter interactions involve the recognition of the core oligosaccharide domain, which contains glucose and heptose [41]. SP-D interacts preferentially with purified LPS molecules characterized by short or absent O-antigens and preferentially agglutinates bacterial strains expressing a predominance of rough (O-antigen-deficient) LPS [41,44].
http://data.bioontology.org/annotator/?text=SP-D%20binds%20to%20purified%20lipopolysaccharide%20%28LPS%29%20isolated%20from%20a%20variety%20of%20Gram-negative%20organisms%20%5B35%2C41%5D.%20In%20addition%2C%20LPS%20is%20the%20major%20cell%20wall%20component%20that%20is%20labeled%20on%20lectin%20blotting%20of%20outer%20membranes%20isolated%20from%20Escherichia%20coli%20%5B41%5D.%20The%20latter%20intera

Rat and human SP-D enhanced the binding and uptake of E. coli K12, Streptococcus pneumoniae, and Staphylococcus aureus by human neutrophils [42]. However, similar effects were observed when the neutrophils were preincubated with SP-D, and there was only a slight enhancement of uptake when bacteria were incubated with human SP-D and washed before their addition to neutrophils. Notably, the extent of binding and internalization was dependent on the extent of multimerization, with human SP-D multimers demonstrating the highest potency. Differences in cell type, the extent of SP-D multimerization, or differences in size or organization of bacterial aggregates could account for some of the apparent inconsistencies.
http://data.bioontology.org/annotator/?text=Rat%20and%20human%20SP-D%20enhanced%20the%20binding%20and%20uptake%20of%20E.%20coli%20K12%2C%20Streptococcus%20pneumoniae%2C%20and%20Staphylococcus%20aureus%20by%20human%20neutrophils%20%5B42%5D.%20However%2C%20similar%20effects%20were%

KeyboardInterrupt: 

#### ---------------------------------------------------
BUILD AND TESTS


### load data

In [ ]:
#example parse directory
parse_directory()

samp_r1=path_files[0]# pmc 
samp_r2=path_files[49] #pdf_json
samp_r1

#load_json_record(samp_r1)
#load_json_record(samp_r2)

In [ ]:
load_json_record(samp_r1)

### See json schemes 

In [ ]:
get_metadata_record(samp_r1) #for pmc_json

In [ ]:
#schema json for pmc_json
{
    'paper_id':'str',
    'metadata': {'title':'str', 'authors':'list'},
    #'abstract': {'text': 'str' ,'section':'str'},
    'bodytext': [{'text': 'str', 'section':'str'}]
}

In [ ]:
get_metadata_record(samp_r2) #for pdf_json

In [ ]:
#eschema json for pdf_json
{
    'paper_id':'str',
    'metadata': {'title':'str', 'authors':'list'},
    'abstract': [{'text': 'str' ,'section':'str'}],
    'bodytext': [{'text': 'str', 'section':'str'}]
}

In [ ]:
#data['paper_id']            ####pmc
#data['metadata']['title']
#data['body_text'] #list of paragraphs

#data['paper_id']            #### pdf_json
#data['metadata']['title']
#data['abstract'] #list of paragraps 
#data['body_text'] #list of paragraps

### Proposal Nosql dbSchemas

In [ ]:
#Nosql schemaDB proposal V.2:  schema COLLECTION of texts annotated from bioportal
list(Item_collection)

In [ ]:
#Implementation Nosql schemaDB proposal V.2
item_collection = {
    'paper_id':'',
    'chunks_paper':[{'text':[], 
                         'section': [], 
                         'annotations':[{'matchTerm':[],
                                         'ontology':[],
                                         'annotatedClass':[]}
                                       ]
                        }]    
} 

In [ ]:
#schema for each chunk of document for annotate
chunks = {'texts':[], 'sections':[], 'annotations': []}

In [ ]:
#schema for results Bioportal api: annotations for each chunk.
annotations = {'matchTerm':[],             
                     'ontology':[],
                     'annotatedClass':[]}

In [ ]:
#implementations of methods for organization and store data
#schema_chunks['text'].append('text_to_annotate')
#schema_chunks['section'].append('section_text_to_annotate')
#schema_chunks['annotations'].append(schema_annotations)
#item_collection['chunks_paper'].append(schema_chunks)
#item_collection['paper_id'] = 'paper_id'

## Tests

### test cero

### call api and retrieve annotations for one fragment of text

In [ ]:
parse_directory()

In [ ]:
#prepared url params
text_to_annotate = load_json_record(path_files[0])['body_text'][0]['text']
text_to_annotate 

#for test param
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"
ontologies

#make url2
URL=make_url_query(text_to_annotate)
URL

#longest_only={true|false}
PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }
#PARAMS

In [ ]:
#load_json_record(path_files[0]).keys()

In [ ]:
#init item_collection schema
item_collection = {
    'paper_id':'',
    'chunks_paper':[]    
} 

call_endpoint_annotator(URL,PARAMS)

#json_api_results
#len(json_api_results)
#json_api_results[0].keys()

chunks = {'texts':[], 'sections':[], 'annotations': []}

# init schema_annotations
annotations= {'matchTerm':[],             
                     'ontology':[],
                     'annotatedClass':[]}

In [ ]:
#process results api V.1
for i in range(len(json_api_results)):
    annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
    annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
    annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
chunks['annotations'].append(annotations)
chunks['texts'].append(text_to_annotate)
chunks['sections'].append(load_json_record(path_files[0])['body_text'][0]['section'])
item_collection['chunks_paper'].append(chunks)
item_collection['paper_id'] += load_json_record(path_files[0])['paper_id']

In [ ]:
test = item_collection.copy()

In [ ]:
#Some Tests for schema annotations process results api v.1

#for marchTerms
test_list_matchTerm = []
for result in json_api_results:
    test_list_matchTerm.append(result['annotations'][0]['text'])
                               
#for list ontology
test_list_ontology = []
for result in json_api_results:
    test_list_ontology.append(result['annotatedClass']['links']['ontology'])
                               
#for annotated class
test_list_annotated_class = []
for result in json_api_results:
    test_list_annotated_class.append(result['annotatedClass']['@id'])

assert annotations['matchTerm'] == test_list_matchTerm
assert annotations['ontology'] == test_list_ontology
assert annotations['annotatedClass'] == test_list_annotated_class

In [ ]:
#test class db
item = dbMain()

In [ ]:
#process results api v.2: employed class db
item.restart_annotations()
for i in range(len(json_api_results)):
    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                            json_api_results[i]['annotatedClass']['links']['ontology'],
                            json_api_results[i]['annotatedClass']['@id']
                           )
item.update_chunks(text_to_annotate,
                         load_json_record(path_files[0])['body_text'][0]['section'],
                         item.annotations
                         )
item.update_item_collection(load_json_record(path_files[0])['paper_id'],
                                  item.chunks)

In [ ]:
#Some Tests for schema annotations process results api v.2

#for marchTerms
test_list_matchTerm = []
for result in json_api_results:
    test_list_matchTerm.append(result['annotations'][0]['text'])
                               
#for list ontology
test_list_ontology = []
for result in json_api_results:
    test_list_ontology.append(result['annotatedClass']['links']['ontology'])
                               
#for annotated class
test_list_annotated_class = []
for result in json_api_results:
    test_list_annotated_class.append(result['annotatedClass']['@id'])

assert annotations['matchTerm'] == item.annotations['matchTerm']
assert annotations['ontology'] == item.annotations['ontology']
assert annotations['annotatedClass'] == item.annotations['annotatedClass']

In [ ]:
item.annotations

In [ ]:
item.chunks

In [ ]:
item.item_collection['chunks_paper']

In [ ]:
test['chunks_paper']

In [ ]:
#test for results belong process results api v.1 v.2
assert test['paper_id'] == item.item_collection['paper_id']
assert test['chunks_paper'] == item.item_collection['chunks_paper']

### Test one

### call api for each fragment of text in one pmc_json record

In [ ]:
# for body text pmc_json
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#init dbs schemas
emty_sections = []
item_collection = {'paper_id':'',
                   'chunks_paper':[]    
                  } 
chunks = {'texts':[], 
          'sections':[], 
          'annotations': []
         }
for idx, _chunk in enumerate(load_json_record(samp_r1)['body_text']):
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init schema annotations
        annotations= {'matchTerm':[],             
                         'ontology':[],
                         'annotatedClass':[]}
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
            annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
            annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
        chunks['texts'].append(_chunk['text'])
        chunks['sections'].append(_chunk['section'])
        chunks['annotations'].append(annotations)
    else:
        emty_sections.append(idx)
        pass
item_collection['paper_id'] += load_json_record(samp_r1)['paper_id']
item_collection['chunks_paper'].append(chunks)

In [ ]:
emty_sections

In [ ]:
test = load_json_record(samp_r1)
test['body_text'][17]

In [ ]:
item_collection['chunks_paper'][0]['texts'][1]

In [ ]:
item_collection['chunks_paper'][0]['sections']

In [ ]:
item_collection['chunks_paper'][0]['annotations']

In [ ]:
# for body text pmc_json
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#init db class
emty_sections = []

item = dbMain()
for idx, _chunk in enumerate(load_json_record(samp_r1)['body_text']):
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init schema annotations
        item.restart_annotations()
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                    json_api_results[i]['annotatedClass']['links']['ontology'],
                                    json_api_results[i]['annotatedClass']['@id']
                                    
                                   )
        item.update_chunks(_chunk['text'],
                           _chunk['section'],
                           item.annotations
                          )
    else:
        emty_sections.append(idx)
        pass
item.update_item_collection(load_json_record(samp_r1)['paper_id'],
                            item.chunks)

In [ ]:
item.item_collection['chunks_paper'][0]['annotations']

### Test two

### call api for each fragment of text in pdf_json record

In [ ]:
#test for pdf_json
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#init dbs schemas
chunks = {'texts':[], 'sections':[], 'annotations': []}
for _chunk in load_json_record(samp_r2)['abstract']:
    
    chunks['texts'].append(_chunk['text'])  # text to annotate
    chunks['sections'].append(_chunk['section'])
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init schema annotations
        annotations= {'matchTerm':[],             
                         'ontology':[],
                         'annotatedClass':[]}
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
            annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
            annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
        chunks['annotations'].append(annotations)
    else:
        pass
    
print('ok abstract')
#Init api params

for _chunk in load_json_record(samp_r2)['body_text']:
    
    chunks['texts'].append(_chunk['text'])  # text to annotate
    chunks['sections'].append(_chunk['section'])
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init schema annotations
        annotations= {'matchTerm':[],             
                         'ontology':[],
                         'annotatedClass':[]}
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
            annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
            annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
        chunks['annotations'].append(annotations)
    else:
        pass

In [ ]:
test_list_sections = []
for chunk_ in load_json_record(samp_r2)['abstract']:
    test_list_sections.append(chunk_['section'])
for chunk_ in load_json_record(samp_r2)['body_text']:
    test_list_sections.append(chunk_['section'])
if load_json_record(samp_r2)['metadata']['title']:
    test_list_sections.append('title')
len(test_list_sections)

In [ ]:
#test for pdf_json
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#init dbs schemas
emty_sections = []
item = dbMain()
for idx,_chunk in enumerate(load_json_record(samp_r2)['abstract']):
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init db schema annotations
        item.restart_annotations()
       
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                    json_api_results[i]['annotatedClass']['links']['ontology'],
                                    json_api_results[i]['annotatedClass']['@id']
                                   )
        item.update_chunks(_chunk['text'],
                           _chunk['section'],
                           item.annotations
                          )
    else:
        emty_sections.append(idx)
    
print('ok abstract')
#Init api params

for idx, _chunk in enumerate(load_json_record(samp_r2)['body_text']):
    
    #make url here
    text_to_annotate = _chunk['text']
    print(text_to_annotate)
    
    if text_to_annotate:
        make_url_query(text_to_annotate)
        print(URL)

        #call api here
        call_endpoint_annotator(URL,PARAMS)
        #init schema annotations
        item.restart_annotations()
       
        #proccess and store resultsfor section here
        for i in range(len(json_api_results)):
            item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                    json_api_results[i]['annotatedClass']['links']['ontology'],
                                    json_api_results[i]['annotatedClass']['@id']
                                   )
        item.update_chunks(_chunk['text'],
                           _chunk['section'],
                           item.annotations
                          )
    else:
        emty_sections.append(idx)
item.update_db_main()

In [ ]:
test_list_sections = []
for chunk_ in load_json_record(samp_r2)['abstract']:
    test_list_sections.append(chunk_['section'])
for chunk_ in load_json_record(samp_r2)['body_text']:
    test_list_sections.append(chunk_['section'])
'''if load_json_record(samp_r2)['metadata']['title']:
    test_list_sections.append('title')'''
len(test_list_sections)

In [ ]:
assert test_list_sections == item.chunks['sections']

### Test tree 

### Call api for each fragment of text in pcm_json and pdf_json records

In [ ]:
db = []

ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#parse_directory()

path_files = [samp_r1,samp_r2]

for idx, path in enumerate(path_files):
    
    item_collection = {
    'paper_id':'',
    'chunks_paper':[]    
    } 
         
    load_json_record(path)
        
    try: 
        
        schema_chunks = {'texts':[], 'sections':[], 'annotations': []}
        for _chunk in load_json_record(path)['abstract']:
    
            schema_chunks['texts'].append(_chunk['text'])  # text to annotate
            schema_chunks['sections'].append(_chunk['section'])
    
            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                schema_annotations= {'matchTerm':[],             
                                 'ontology':[],
                                 'annotatedClass':[]}
            #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    schema_annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
                    schema_annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
                    schema_annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
                schema_chunks['annotations'].append(schema_annotations)
            else:
                pass
    
        print('ok abstract')
   

        for _chunk in load_json_record(path)['body_text']: # fragments of text in body
    
            schema_chunks['texts'].append(_chunk['text'])  # text to annotate
            schema_chunks['sections'].append(_chunk['section'])

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)
    
            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                schema_annotations= {'matchTerm':[],             
                                 'ontology':[],
                                 'annotatedClass':[]}
                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    schema_annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
                    schema_annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
                    schema_annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
                schema_chunks['annotations'].append(schema_annotations)
            else:
                pass
            
    except:
        schema_chunks = {'texts':[], 'sections':[], 'annotations': []}
        for _chunk in load_json_record(path)['body_text']:

            schema_chunks['texts'].append(_chunk['text'])  # text to annotate
            schema_chunks['sections'].append(_chunk['section'])

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                schema_annotations= {'matchTerm':[],             
                                 'ontology':[],
                                 'annotatedClass':[]}
                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    schema_annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
                    schema_annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
                    schema_annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
                schema_chunks['annotations'].append(schema_annotations)
            else:
                pass
                
    finally:
        #for title
        text_to_annotate = json_record['metadata']['title']
        
        if text_to_annotate:
            make_url_query(text_to_annotate)
            print(URL)

            #call api here
            call_endpoint_annotator(URL,PARAMS)
            #init schema annotations
            schema_annotations= {'matchTerm':[],             
                                 'ontology':[],
                                 'annotatedClass':[]}
            #proccess and store resultsfor section here
            for i in range(len(json_api_results)):
                schema_annotations['matchTerm'].append(json_api_results[i]['annotations'][0]['text'])
                schema_annotations['ontology'].append(json_api_results[i]['annotatedClass']['links']['ontology'])
                schema_annotations['annotatedClass'].append(json_api_results[i]['annotatedClass']['@id'])
            schema_chunks['annotations'].append(schema_annotations)
            schema_chunks['sections'].append('title')
            schema_chunks['texts'].append(text_to_annotate)
        item_collection['paper_id'] += json_record['paper_id'] 
        item_collection['chunks_paper'].append(schema_chunks)
        print('paper index:',idx,'processed')
        db.append(item_collection)

In [ ]:
db[1]

In [15]:
os.listdir('.')

['.ipynb_checkpoints',
 '.git',
 'README.md',
 'SectionPaperAnnotator.ipynb',
 'data']

In [ ]:
parse_directory()

In [21]:
samp_r1 = path_files[0]
samp_r2 = path_files[48]

In [22]:
ontologies = "MESH,COVID-19,HPIO,SNOMEDCT,OCHV,PREMEDONTO"

PARAMS = {'apikey':API_KEY,'ontologies':ontologies,
          'expand_class_hierarchy':'false','class_hierarchy_max_level':0,
           'longest_only':'true'
         }

#parse_directory()

path_files = [samp_r1,samp_r2]

for idx, path in enumerate(path_files):
    
    load_json_record(path)
    emty_sections = []
    item = dbMain()
        
    try: 
        
        for idx, _chunk in enumerate(json_record['abstract']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init db schema annotations
                item.restart_annotations()

                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']
                                           )
                print(item.annotations)
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
    
            else:
                emty_sections.append(idx)

        print('ok abstract')
        #Init api params

        for idx, _chunk in enumerate(json_record['body_text']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                item.restart_annotations()

                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']
                                           )
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
            else:
                emty_sections.append(idx)
                
    except KeyError:

        for idx, _chunk in enumerate(json_record['body_text']):

            #make url here
            text_to_annotate = _chunk['text']
            print(text_to_annotate)

            if text_to_annotate:
                make_url_query(text_to_annotate)
                print(URL)

                #call api here
                call_endpoint_annotator(URL,PARAMS)
                #init schema annotations
                item.restart_annotations()
                #proccess and store resultsfor section here
                for i in range(len(json_api_results)):
                    item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                            json_api_results[i]['annotatedClass']['links']['ontology'],
                                            json_api_results[i]['annotatedClass']['@id']

                                           )
                item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
            else:
                emty_sections.append(idx)

    finally:
        #for title
        text_to_annotate = json_record['metadata']['title']
        
        if text_to_annotate:
            make_url_query(text_to_annotate)
            print(URL)

            #call api here
            call_endpoint_annotator(URL,PARAMS)
            #init schema annotations
            
            item.restart_annotations()
            #proccess and store resultsfor section here
            for i in range(len(json_api_results)):
                item.update_annotations(json_api_results[i]['annotations'][0]['text'],
                                        json_api_results[i]['annotatedClass']['links']['ontology'],
                                        json_api_results[i]['annotatedClass']['@id']

                                           )
            item.update_chunks(_chunk['text'],
                                   _chunk['section'],
                                   item.annotations
                                  )
        item.update_item_collection(json_record['paper_id'], item.chunks)
        print('paper index:',idx,'processed')
        item.update_db_main()

Mycoplasma pneumoniae is a common cause of upper and lower respiratory tract infections. It remains one of the most frequent causes of atypical pneumonia particularly among young adults.[1,2,3,4,5] Although it is highly transmissible, most infections caused by this organism are relatively minor and include pharyngitis, tracheobronchitis, bronchiolitis, and croup with one fifth of infections being asymptomatic.[6,7] Only 3 -10% of infected subjects develop symptoms consistent with bronchopneumonia and mortality from infection is rare.[6,7] The organism is fastidious and difficult to grow on cultures. Therefore, diagnosis of infections caused by this organism is usually confirmed with serological tests or polymerase chain reaction-gene amplification techniques. At King Abdulaziz University Hospital (KAUH), Jeddah, Saudi Arabia, the facility to perform Mycoplasma culture has been available since January 1997. As published information concerning M. pneumoniae infections in Saudi Arabia is 

Outcome of patients with M. pneumoniae infection was classified into 4 categories; uneventful recovery, recovery following complications, death due to M. pneumoniae infection, or death unrelated to M. pneumoniae infection.
http://data.bioontology.org/annotator/?text=Outcome%20of%20patients%20with%20M.%20pneumoniae%20infection%20was%20classified%20into%204%20categories%3B%20uneventful%20recovery%2C%20recovery%20following%20complications%2C%20death%20due%20to%20M.%20pneumoniae%20infection%2C%20or%20death%20unrelated%20to%20M.%20pneumoniae%20infection.
The Statistical Package for Social Sciences (SPSS) program was used for data analysis. Comparison of categorical data was by Chi-square statistic or Fisher's exact test for small expected values.
http://data.bioontology.org/annotator/?text=The%20Statistical%20Package%20for%20Social%20Sciences%20%28SPSS%29%20program%20was%20used%20for%20data%20analysis.%20Comparison%20of%20categorical%20data%20was%20by%20Chi-square%20statistic%20or%20Fisher%

Mycoplasma pneumoniae is one of the most common causes of atypical pneumonia accounting for 5-23% of community-acquired pneumonia,[1,2,3,4,5] In a study of 511 children with acute respiratory tract infection in Riyadh, Saudi Arabia, Mycoplasma pneumoniae was found to be the second most common causative agent after Respiratory syncytial virus (RSV) accounting for 9% of all cases,[8] In a study of 112 adult patients with community acquired pneumonia admitted to a military hospital in Riyadh, Saudi Arabia, this organism accounted for 6% of all cases,[9] In another retrospective study of 567 pneumonic episodes in adult patients from Al-Qassim area, the organism accounted for 23% of all episodes,[10] The organism also causes other relatively minor infections such as pharyngitis, tracheobronchitis, bronchiolitis, and croup. It is transmitted from person-to-person by infected respiratory droplets during close contact. It is most common in school-aged children, military recruits, and college s

More than half (56.5%) of the patients with pneumonia had uneventful recovery. Mortality from M. pneumoniae pneumonia was high (12.5%) and occurred only in patients with underlying comorbidities. None of the 9 patients with no underlying comorbidities died of M pneumoniae pneumonia. The relatively high complications rate (16.7%) and mortality (12.5%) related to M. pneumoniae pneumonia are likely due to selection bias as most patients with pneumonia were sick enough to require admission to the hospital (21/24 or 87.5%) and most of them had comorbidities (20/24 or 83.3%).
http://data.bioontology.org/annotator/?text=More%20than%20half%20%2856.5%25%29%20of%20the%20patients%20with%20pneumonia%20had%20uneventful%20recovery.%20Mortality%20from%20M.%20pneumoniae%20pneumonia%20was%20high%20%2812.5%25%29%20and%20occurred%20only%20in%20patients%20with%20underlying%20comorbidities.%20None%20of%20the%209%20patients%20with%20no%20underlying%20comorbidities%20died%20of%20M%20pneumoniae%20pneumonia.%2

{'matchTerm': ['PATIENTS', 'PATIENTS', 'PATIENTS', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'POSITIVE', 'CULTURES', 'SPECIMENS', '1997', '1998', 'IDENTIFIED', 'IDENTIFIED', 'IDENTIFIED', 'IDENTIFIED', 'MICROBIOLOGY', 'MICROBIOLOGY', 'MICROBIOLOGY', 'MICROBIOLOGY', 'RECORDS', 'RECORDS', 'CHARTS', 'CHARTS', 'REVIEWED', 'REVIEWED'], 'ontology': ['http://data.bioontology.org/ontologies/MESH', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/PREMEDONTO', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/PREMEDONTO', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/SNOMEDCT', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/SNOMEDCT', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/SNOMEDCT', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.

{'matchTerm': ['PUBLISHED', 'PUBLISHED', 'DATA', 'EXCEPT FOR', 'EXCEPT FOR', 'EXCEPT FOR', 'FINDING', 'FINDING', 'FINDING', 'INFECTIONS', 'INFECTIONS', 'COMMON', 'COMMON', 'COMMON', 'COMMON', 'INFANTS', 'INFANTS', 'PRESCHOOL CHILDREN', 'MORTALITY RATE', 'MORTALITY RATE', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PNEUMONIA', 'PATIENTS', 'PATIENTS', 'PATIENTS', 'COMORBIDITIES', 'HIGH', 'HIGH', 'HIGH', 'HIGH'], 'ontology': ['http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/PREMEDONTO', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/SNOMEDCT', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.bioontology.org/ontologies/PREMEDONTO', 'http://data.bioontology.org/ontologies/MESH', 'http://data.bioontology.org/ontologies/SNOMEDCT', 'http://data.bioontology.org/ontologies/MESH', 'http://data.bioontology.org/ontologies/OCHV', 'http://data.b

M. pneumoniae was cultured using the classic M. pneumoniae agar medium (M.P. agar) and the Pneumofast tray (Pneumofast ® , International Microbio, Signes, France). Specimens were processed according to the instructions of the manufacturer. The M.P. agars and Pneumofast trays were incubated anaerobically at 37°C and inspected daily for 4 weeks. The organism was identified based on typical colonial morphology (granular colonies, rarely fried-egg-like, 10-150 ∝ in diameter) on the M.P. agar medium and the change in the Pneumofast broth color from red to orange then to yellow (glucose fermentation) in the absence of turbidity of the broth. Antibiotic sensitivity profile on the Pneumofast tray was also used for identification according to the instructions of the manufacturer. Bacterial and viral cultures were performed using standard methods.
http://data.bioontology.org/annotator/?text=M.%20pneumoniae%20was%20cultured%20using%20the%20classic%20M.%20pneumoniae%20agar%20medium%20%28M.P.%20aga

Clinical manifestations associated with M. pneumoniae infections are summarized in Table 2 . Pneumonia was more common than upper respiratory tract infections (57.5 % versus 27.5%, respectively). Immunocompromised patients were more likely to present with pneumonia as opposed to upper respiratory tract infection or bronchiolitis than non-immunocompromised patients (8/9 versus 16/31, P = 0.05). Similarly, there was a tendency for patients 60 years of age or older to present with pneumonia more frequently than those below 60 (4/4 versus 20/36, P = 0.1). Of the 24 patients with clinically or radiologically confirmed pneumonia, 19 (79.2%) had crepitations and only 6 (25%) had bronchial breath sounds on physical examination. Of the 16 patients in whom wheezes were detected, 9 (56.3%) were not known to have asthma or other obstructive airway disease. Table 3 . Of the 24 patients with pneumonia, 21 (87.5%) were admitted to the hospital, and 20 (83.3%) had comorbidities. All patients with uppe

More than one half of patients (57.5%) presented with pneumonia, and about a third (27.5%) presented with upper respiratory tract infection, Immunocompromised patients and patients 60 years of age or older were more likely to present with pneumonia as opposed to upper respiratory tract infection than non-immunocompromised patients or those below 60 years of age. Cough (82.5%), fever (75%), and malaise (58.8%) were the most common presenting symptoms. Cough was usually dry or slightly productive of white sputum and mild to moderate in severity. Most febrile patients had mild to mod- erate fever of 39°C or less; high-grade fever of more than 39°C was rare. Crepitations (60%), and wheezes (40%) were the most common signs. Wheezes were as common in patients with no history of obstructive airway disease (9 patients) as it was in those with such a history (7 patients). Bronchial breathing as a sign of consolidation was detected in only one fourth of patients with pneumonia, which is consiste

In [29]:
# make some test her
one_item = dbMain.db[0]
one_item.keys()

dict_keys(['paper_id', 'chunks_paper'])

In [32]:
one_item['chunks_paper'][0].keys()

dict_keys(['texts', 'sections', 'annotations'])

In [53]:
one_item['chunks_paper'][0]['texts'][0]

'Mycoplasma pneumoniae is a common cause of upper and lower respiratory tract infections. It remains one of the most frequent causes of atypical pneumonia particularly among young adults.[1,2,3,4,5] Although it is highly transmissible, most infections caused by this organism are relatively minor and include pharyngitis, tracheobronchitis, bronchiolitis, and croup with one fifth of infections being asymptomatic.[6,7] Only 3 -10% of infected subjects develop symptoms consistent with bronchopneumonia and mortality from infection is rare.[6,7] The organism is fastidious and difficult to grow on cultures. Therefore, diagnosis of infections caused by this organism is usually confirmed with serological tests or polymerase chain reaction-gene amplification techniques. At King Abdulaziz University Hospital (KAUH), Jeddah, Saudi Arabia, the facility to perform Mycoplasma culture has been available since January 1997. As published information concerning M. pneumoniae infections in Saudi Arabia is

In [54]:
one_item['chunks_paper'][0]['sections'][0]

'Introduction'

In [55]:
one_item['chunks_paper'][0]['annotations'][0]

{'matchTerm': ['MYCOPLASMA PNEUMONIAE',
  'MYCOPLASMA PNEUMONIAE',
  'MYCOPLASMA PNEUMONIAE',
  'MYCOPLASMA PNEUMONIAE',
  'IS A',
  'COMMON',
  'COMMON',
  'COMMON',
  'COMMON',
  'CAUSE',
  'CAUSE',
  'UPPER',
  'UPPER',
  'UPPER',
  'LOWER RESPIRATORY TRACT INFECTIONS',
  'FREQUENT',
  'FREQUENT',
  'FREQUENT',
  'FREQUENT',
  'ATYPICAL PNEUMONIA',
  'ATYPICAL PNEUMONIA',
  'ATYPICAL PNEUMONIA',
  'YOUNG',
  'YOUNG',
  'YOUNG',
  'ADULTS',
  'ADULTS',
  'HIGHLY',
  'INFECTIONS',
  'INFECTIONS',
  'ORGANISM',
  'ORGANISM',
  'ORGANISM',
  'ORGANISM',
  'ORGANISM',
  'ORGANISM',
  'ORGANISM',
  'MINOR',
  'MINOR',
  'MINOR',
  'MINOR',
  'MINOR',
  'MINOR',
  'INCLUDE',
  'INCLUDE',
  'PHARYNGITIS',
  'PHARYNGITIS',
  'PHARYNGITIS',
  'PHARYNGITIS',
  'PHARYNGITIS',
  'TRACHEOBRONCHITIS',
  'TRACHEOBRONCHITIS',
  'TRACHEOBRONCHITIS',
  'BRONCHIOLITIS',
  'BRONCHIOLITIS',
  'BRONCHIOLITIS',
  'CROUP',
  'CROUP',
  'CROUP',
  'CROUP',
  'FIFTH',
  'ASYMPTOMATIC',
  'ASYMPTOMATIC',
  'AS

In [72]:
item.store_db_main()

In [ ]:
len(item.db[1]['chunks_paper'][0]['annotations'][0]['annotatedClass'])

In [ ]:
load_json_record(samp_r1)['paper_id']

In [ ]:
samp_r2

###  FINAL CODE TO ANNOTATED PAPERS

In [ ]:
len(item.db)